In [255]:
from enum import Enum
from random import shuffle
import numpy as np
from operator import mul
import random
import termcolor
import pandas as pd

class Suit(Enum):
    SPADE = '♠'
    CLUB = '♣'
    HEART = '♡'
    DIAMOND = '♢'
    def __str__(self):
        return self.value
    def __repr__(self):
        return f"Suit.{self.name}"


class Number(Enum):
    ACE = (1, 'A')
    TWO = (2, '2')
    THREE = (3, '3')
    FOUR = (4, '4')
    FIVE = (5, '5')
    SIX = (6, '6')
    SEVEN = (7, '7')
    EIGHT = (8, '8')
    NINE = (9, '9')
    TEN = (10, '10')
    JACK = (11, 'J')
    QUEEN = (12, 'Q')
    KING = (13, 'K')

    def __init__(self, val, string):
        self.val = val
        self.string = string

    def __str__(self):
        return self.string

    def __repr__(self):
        return f"Number.{self.name}"
    
class Card:
    def __init__(self, suit, number):
        if not (isinstance(suit, Suit) and isinstance(number, Number)):
            raise ValueError  # Enum じゃないとエラー
        self.suit = suit
        self.number = number

    def __str__(self):
        return str(self.suit) + str(self.number)

    def __repr__(self):
        return f"Card({self.__str__()})"
    
    def __eq__(self, other):
        return (self.suit, self.number) == (other.suit, other.number)
            
    
class Hand(list):
    def __init__(self,card_list):
        super().__init__(
            i for i in card_list
        )
        
    def check_number(self):
        number_list=[i.number.val for i in self]
        return number_list
    
    def check_suit(self):
        suit_list=[str(i.suit) for i in self]
        return suit_list
    
    def choice(self,card):
        #Card(Suit.SPADE, Number.ACE)
        if card in self:
            self.remove(card)
            return card
        else:
            raise ValueError
            
    def check(self,card):
        return card in self
    

class Deck(list):
    def __init__(self):
        super().__init__(
            Card(suit, number) for suit in Suit for number in Number
        )  # list の初期化を呼び出す
        self.shuffle()  # 最初にシャッフル
    def shuffle(self):
        shuffle(self)
    def draw(self):
        return self.pop()
    def deal(self, players_num):
        cards=[Hand(i) for i in np.array_split(self,players_num)]
        self.clear()
        return cards

In [256]:
# ゲームの状態
class State:
    # 初期化
    def __init__(self, players_num=3,field_cards=None, players_cards=None,turn_player=None,pass_count=None,out_player=None,all_actions=None,all_players=None):
        if players_cards==None:
            deck = Deck()
            self.players_cards=deck.deal(players_num)
            self.players_num=players_num
            self.field_cards=np.zeros((4,13), dtype='int64')
            self.start_flags=[0]*self.players_num
            self.pass_count=[0]*self.players_num
            self.out_player=[]
            self.all_actions = []
            self.all_players = []
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
            for players_number in range(players_num):
                self.start_flags[players_number]=self.choice_seven(hand=self.players_cards[players_number])
            self.turn_player=self.start_flags.index(1)
        else:
            self.players_cards=players_cards
            self.field_cards=field_cards
            self.players_num=players_num
            self.turn_player=turn_player
            self.pass_count=pass_count
            self.out_player=out_player
            self.all_actions=all_actions
            self.all_players =all_players
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
        

    #7のカードを出す
    def choice_seven(self,hand):
        start_flag=0
        for card in [Card(suit,Number.SEVEN) for suit in Suit]:
            if hand.check(Card(Suit.DIAMOND,Number.SEVEN))==True:
                start_flag=1
            if hand.check(card)==True:
                self.put_card(hand.choice(card))
        return start_flag
    
    def choice_card(self,hand,card):
        hand.choice(card)
    
    #場にカードを出す
    def put_card(self,card):
        num=10
        for s,i in zip(Suit,range(4)):
            if card.suit==s:
                num=i
        #state.my_hands().remove(card)
        self.field_cards[num][card.number.val-1]=int(1)
    
    
    # 場で出せる手のリスト取得
    # 3パスの人がいた時、未対応
    def legal_actions(self):
        actions = []
        for suit,n in zip(Suit,range(4)):
            
            if self.field_cards[n][0:6][::-1].tolist().count(1)!=6:
                actions.append(Card(suit,self.num_to_Enum(6-self.field_cards[n][0:6][::-1].tolist().index(0))))
                
            if self.field_cards[n][7:13].tolist().count(1)!=6:
                actions.append(Card(suit,self.num_to_Enum(8+self.field_cards[n][7:13].tolist().index(0))))
        return actions
    
    
    # 自分が出せる手のリスト取得
    def my_actions(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return actions
    def my_actions_str(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return [str(i) for i in actions]
    
    # 自分の手札取得
    def my_hands(self):
        return self.players_cards[self.turn_player]
    def my_hands_str(self):
        return [str(i) for i in self.players_cards[self.turn_player]]
            
    
    def num_to_Enum(self,num):
        enum_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR, 
                   Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                   Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                   Number.KING]
        return enum_list[num-1]
    
    
    # 次の状態の取得
    def next(self, action,pass_flag = 0):
        if self.my_actions()==[]:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        elif pass_flag == 1:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        else:
            self.players_cards[self.turn_player].remove(action)
            self.put_card(action)
            self.all_actions.append(action)
            self.all_players.append(self.turn_player)
            
        #次のプレイヤーに
        self.next_player() 
        return State(players_num=self.players_num,field_cards=self.field_cards, players_cards=self.players_cards,turn_player=self.turn_player,pass_count=self.pass_count,out_player=self.out_player,all_actions=self.all_actions,all_players=self.all_players)
   
    #次のプレイヤーの取得 
    def next_player(self):
        flag=0
        while flag==0:
            if self.turn_player+1>=self.players_num:
                self.turn_player=self.turn_player+1-self.players_num
            else:
                self.turn_player+=1

            if self.turn_player not in self.out_player:
                flag=1
    
    #パスの上限判定
    def pass_check(self):
        out_list=self.out_player
        if self.pass_count[self.turn_player]>3:
            for card in self.my_hands():
                self.put_card(card)
                self.all_actions.append(card)
                self.all_players.append(self.turn_player)
                
            out_list.append(self.turn_player)
            
            self.out_player=out_list
            
    def to_str(self,num):
        return str(num)
    
    #勝ち負け判定
    def is_done(self):
        return len(self.my_hands())==0
        
            
    # 状態表示
    def __str__(self):
        str = ''
        field_cards=self.field_cards.tolist()
        out_list=[list(map(mul,self.all_cards[i],field_cards[i])) for i in range(4)]
        str += "場のカード\n\n"
        for i in range(len(out_list)):
            minilist=out_list[i]
            for j in range(len(minilist)):
                if minilist[j] == "":
                    str += " -- "
                else:
                    str +=" "+minilist[j]+" "
            str += '\n'
        num=self.to_str(self.turn_player)
        pass_cnt=self.to_str(self.pass_count[self.turn_player])
        str+="\nプレイヤー"+num+"番　　パス回数"+pass_cnt+"\n"
        str += "\nあなたの手札\n"
        
        out_list=self.my_hands_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "
            
        str += "\n\n出せるカード\n"
        
        out_list=self.my_actions_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "
        
        str += "\n"
        
        return str    

In [257]:
def num_to_Card(number,suit):
    number_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR, 
                Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                Number.KING]
    suit_list=[Suit.SPADE,Suit.CLUB,Suit.HEART,Suit.DIAMOND]
    return Card(suit_list[suit],number_list[number-1])

In [258]:
# ランダム行動 AI
def random_action(state):
    my_actions = state.my_actions()
    if my_actions != []:
        return my_actions[random.randint(0, len(my_actions)-1)]
    else:
        my_actions=[]
        return my_actions

In [259]:
#ライブラリを追加で使用したい場合はここでimportして下さい。

MY_PLAYER_NUM = 0

import random

# Blackjack風のカウント表
#  2,3,4,5,6 => +1
#  7,8,9     =>  0
#  10,J,Q,K,A => -1
def blackjack_count_value(card_number_val):
    """
    カードの数字 (1~13) を受け取り、ブラックジャックのようなカウント値を返す。
    A=1, J=11, Q=12, K=13
    """
    if 2 <= card_number_val <= 6:
        return +1
    elif 7 <= card_number_val <= 9:
        return 0
    else:
        # 10, 11(J), 12(Q), 13(K), 1(A)
        return -1

def my_AI(state):
    # """
    # 自分のAIロジックを実装し、(action, pass_flag) を返します。
    # """
    # def DistFrom7(hand):
    #     # カードの番号から7との距離を計算
    #     return abs(hand.number.val - 7)

    # # 出せるカードを優先度順にソート（7に近い順）
    # hands_priority = sorted(state.my_actions(), key=DistFrom7, reverse=False)

    # if hands_priority:
    #     # 出せるカードがある場合、最優先のカードを選択
    #     action = hands_priority[0]
    #     pass_flag = 0  # パスしない
    # else:
    #     # 出せるカードがない場合、パス
    #     action = None  # パスを示す値
    #     pass_flag = 1  # パスフラグをセット

    # return action, pass_flag
    
    """
    戦略的パスを行うAI。
      1. 全カード52枚から、自分の手札+すでに場に出たカードを除いた「不明カード」を推定。
      2. 不明カードに対してブラックジャックカウントを行い、「相手が強いカードを持っていそう」ならパスを検討。
      3. 実際に出すカードは、評価値(以下best_actionに準ずる)がもっとも高いカードを選択。
      4. ただしパス回数が3回未満の場合のみパス可能。
    """

    # ------------------------------
    # 0. パス判定用の変数
    # ------------------------------
    pass_flag = 0  # 0: パスしない, 1: パスする

    # ------------------------------
    # 1. 自分が出せるカードを取得
    # ------------------------------
    my_actions = state.my_actions()
    if not my_actions:
        # 出せるカードが無い場合は強制的にパス扱い
        return [], 0

    # ------------------------------
    # 2. ブラックジャックカウンティング:
    #    - 52枚から自分の手札 + 場に出たカード(all_actions)を除いた残りを数える
    # ------------------------------
    from collections import Counter
    
    # 全52枚のリストを生成
    all_deck = []
    from enum import Enum
    # すでに定義済みの Suit, Number, Card を利用しますが、念のため再定義を避ける
    # -> ここでは state.all_cards などを活用 or 自前で作る
    suit_list = [0,1,2,3]  # 0:♠,1:♣,2:♡,3:♢
    num_list = list(range(1,14))  # 1~13 (A~K)
    
    # 52枚のCard(suit, number)を作る
    from copy import deepcopy
    deck_temp = []
    for s in suit_list:
        for n in num_list:
            deck_temp.append(num_to_Card(n, s))  # num_to_Cardはノートブック内関数
    
    # 自分の手札と、場に出たカードを除外
    #  (1) 自分の手札
    my_hand_cards = state.my_hands()
    #  (2) すでに場に出た全カード
    used_cards = state.all_actions
    
    deck_unknown = []
    for c in deck_temp:
        if (c not in my_hand_cards) and (c not in used_cards):
            deck_unknown.append(c)
    
    # この未知カードをブラックジャック風にカウント
    #   count > 0 => 低いカードが多い => 攻めやすい
    #   count < 0 => 高いカードが多い => 相手が強そう => パスしたい
    bj_count = 0
    for c in deck_unknown:
        bj_count += blackjack_count_value(c.number.val)

    # ------------------------------
    # 3. 簡易的パス判定ロジック
    #    - bj_count < 0 かつ まだパス回数が3回に達していないならパス
    # ------------------------------
    my_pass_count = state.pass_count[state.turn_player]
    if bj_count < 0 and my_pass_count < 3:
        # 攻めると危険そうなので、あえてパスしてみる
        pass_flag = 1
    
    # ただし、本当に出せるカードが無ければ pass_flag = 0 でもパス扱いされるので注意。
    # （あくまで「出せるカードがあるけどパスしたい」場合のみ pass_flag=1 をセット）
    
    # ------------------------------
    # 4. 出すカードの評価: best_action 相当
    #    (ここではユーザーが提示したコードを基に多少省略)
    # ------------------------------
    if pass_flag == 0:
        # パスしないときのみ、「最良スコアのカード」を選択
        # ここでは既存のロジックを簡易的に埋め込み
        suit_to_index = {'♠': 0, '♣': 1, '♡': 2, '♢': 3}
        best_score = float('-inf')
        best_cards = []
        
        # スートの枚数カウント
        suit_counts = {'♠': 0, '♣': 0, '♡': 0, '♢': 0}
        for card in state.my_hands():
            suit_counts[card.suit.value] += 1
        
        for card in my_actions:
            suit = card.suit.value
            suit_index = suit_to_index[suit]
            number_index = card.number.val - 1
            score = 0
            
            # A(1)やK(13)をやや高評価
            if number_index == 0 or number_index == 12:
                score += 5
            
            # 隣のカード判定 (6未満,8より大)
            next_indices = []
            if number_index < 6:
                next_indices.append(number_index - 1)
            elif number_index > 6:
                next_indices.append(number_index + 1)
            
            for nxt in next_indices:
                if 0 <= nxt <= 12:
                    if state.field_cards[suit_index][nxt] == 1:
                        score += 2
                    else:
                        score -= 3
                        # 自分が次のカードを持っていれば少し補正
                        have_next = any(
                            c2 for c2 in state.my_hands()
                            if c2.suit.value == suit and c2.number.val -1 == nxt
                        )
                        if have_next:
                            score += 2
            
            # スートを多く持っているほど優先
            score += suit_counts[suit] * 0.5
            
            # 手札を減らすメリット
            score += (len(state.my_hands()) - 1) * 0.1
            
            # 次のカードを出せる可能性
            potential_new_moves = 0
            for c2 in state.my_hands():
                if c2.suit.value == suit:
                    c_idx = c2.number.val - 1
                    if (number_index < 6 and c_idx == number_index -1) or \
                       (number_index > 6 and c_idx == number_index +1):
                        potential_new_moves += 1
            score += potential_new_moves * 1.5
            
            # 最良カード更新
            if score > best_score:
                best_score = score
                best_cards = [card]
            elif score == best_score:
                best_cards.append(card)
        
        # 同点ならランダムに
        chosen_card = random.choice(best_cards)
        return chosen_card, pass_flag
    
    else:
        # pass_flag == 1 の場合、出せるカードはあるけどパスを選ぶ
        return [], 1


In [260]:
# ランダムAIと対戦
state = State()
# ゲーム終了までのループ
while True:
    # ゲーム終了時
    if state.is_done():
        print("勝者 プレイヤー"+str(state.turn_player)+"番")
        break;
        
    pass_flag = 0
    # 行動の取得
    if state.turn_player == MY_PLAYER_NUM:
        action,pass_flag = my_AI(state)
        print(termcolor.colored(state, 'red'))
    else:
        action = random_action(state)
        print(state)

    # 次の状態の取得
    if pass_flag == 1:
        state = state.next(action,pass_flag)
    else:
        state = state.next(action)

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー2番　　パス回数0

あなたの手札
♣6 ♣10 ♡Q ♡4 ♣9 ♢8 ♠10 ♠5 ♠J ♣A ♡6 ♠K ♣2 ♠8 ♣3 

出せるカード
♠8 ♣6 ♡6 ♢8 

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  ♣6  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー0番　　パス回数0

あなたの手札
♡8 ♠3 ♣J ♡9 ♡2 ♢3 ♣K ♠4 ♢9 ♠6 ♢2 ♡10 ♡A ♢J ♢4 ♠A 

出せるカード
♠6 ♡8 

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  ♣6  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー1番　　パス回数0

あなたの手札
♢6 ♣Q ♣5 ♡3 ♢A ♢K ♢Q ♠9 ♡J ♣8 ♡K ♢5 ♡5 ♠2 ♠Q ♢10 ♣4 

出せるカード
♣5 ♣8 ♢6 

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --

In [261]:
# ランダムAIとの勝率チェック
EP_GAME_COUNT = 2000  # 1評価あたりのゲーム数

def player_point(ended_state):
    """ゲーム終了時にプレイヤー0が勝利していれば1ポイント"""
    if ended_state.turn_player == 0:
        return 1
    return 0

def play(next_actions):
    """1ゲームをプレイし、勝者に応じたポイントを返す"""
    state = State()
    while True:
        if state.is_done():
            break
        pass_flag = 0
        if state.turn_player == MY_PLAYER_NUM:
            action, pass_flag = my_AI(state)
        else:
            action = random_action(state)
        # 次の状態の取得
        if pass_flag == 1:
            state = state.next(action, pass_flag)
        else:
            state = state.next(action)
    return player_point(state)

# 任意のアルゴリズムの評価
def evaluate_algorithm_of(label, next_actions):
    """アルゴリズムを評価し、勝利回数と勝率を出力"""
    total_point = 0
    win_count = 0  # プレイヤー0の勝利回数

    for i in range(EP_GAME_COUNT):
        result = play(next_actions)
        total_point += result
        if result == 1:  # 勝利した場合
            win_count += 1
        print('\rEvaluate {}/{} - Wins: {}'.format(i + 1, EP_GAME_COUNT, win_count), end='')
    print('')

    # 平均ポイントの計算
    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))
    print(f"Total Wins: {win_count}/{EP_GAME_COUNT} ({(win_count / EP_GAME_COUNT) * 100:.2f}%)")

# VSランダム
next_actions = (random_action, random_action)
evaluate_algorithm_of('VS_Random {:.3f}', next_actions)


Evaluate 2000/2000 - Wins: 1681
VS_Random 0.841
Total Wins: 1681/2000 (84.05%)


## 決勝大会用にいくつかの追加の機能が実装されています。

### カードが出せるときのPASS
カードを持っているけど、戦略的にpassをしたい時用です。

In [262]:
state = State()

In [263]:
state = state.next(random_action(state),pass_flag=1)

In [264]:
print(state)

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー2番　　パス回数0

あなたの手札
♡10 ♡6 ♣J ♠5 ♡2 ♢10 ♢A ♣8 ♣3 ♣Q ♠4 ♣5 ♠Q ♡3 ♢8 ♡5 

出せるカード
♣8 ♡6 ♢8 



In [265]:
#パス回数のカウント
state.pass_count

[0, 1, 0]

In [266]:
#今のプレイヤーの番号
state.turn_player

2

In [267]:
#今のプレイヤーのパス回数
state.pass_count[state.turn_player]

0

### 今まで出たカードと、そのカードを出したプレイヤー
対戦相手の手札がどのような物か推測できる可能性があります

In [268]:
#今までのデータカード
state.all_actions

[]

In [269]:
#それを出したプレイヤー
state.all_players

[]

In [270]:
def cal_player_num(n):
    out_num = MY_PLAYER_NUM + n
    if out_num > 3:
        out_num = out_num-3
    if out_num < 0:
        out_num = out_num+3
    return out_num

In [271]:
cal_player_num(0)

0

In [272]:
c.suit == Suit.SPADE

NameError: name 'c' is not defined

In [ ]:
count_h_before = 0
for c,p in zip(state.all_actions,state.all_players):
    if p == cal_player_num(-1):
         if c.suit == Suit.HEART:
            count_h_before +=1

In [ ]:
count_h_before

3

## 自分の出せるカードから特定のマークのものを探す

In [ ]:
state = State()

In [ ]:
state.my_actions()

[Card(♠8), Card(♣8), Card(♡8), Card(♢8)]

In [ ]:
my_a = state.my_actions()

In [ ]:
[a.suit for a in my_a]

[Suit.SPADE, Suit.CLUB, Suit.HEART, Suit.DIAMOND]

In [ ]:
suit = [a.suit for a in my_a]

In [ ]:
idx = suit.index(Suit.CLUB) 
idx

1

In [ ]:
state.my_actions()[idx]

Card(♣8)

## 手札を並び替える(復習)

In [ ]:
state.my_hands().check_number()

[13, 13, 2, 1, 8, 2, 5, 10, 8, 1, 1, 3, 4, 8, 8, 11]

In [ ]:
def DistFrom7(hand):
    return abs(hand.number.val)

hands_sorted = sorted(state.my_hands(), key = DistFrom7, reverse = False)
print(state.my_hands())
print(hands_sorted)

[Card(♡K), Card(♣K), Card(♠2), Card(♣A), Card(♢8), Card(♣2), Card(♠5), Card(♢10), Card(♡8), Card(♢A), Card(♠A), Card(♡3), Card(♡4), Card(♠8), Card(♣8), Card(♢J)]
[Card(♣A), Card(♢A), Card(♠A), Card(♠2), Card(♣2), Card(♡3), Card(♡4), Card(♠5), Card(♢8), Card(♡8), Card(♠8), Card(♣8), Card(♢10), Card(♢J), Card(♡K), Card(♣K)]


In [ ]:
state=State()

In [ ]:
#自分が出せるカードを並び替える場合

action_sorted = sorted(Hand(state.my_actions()), key = DistFrom7, reverse = False)
print(state.my_actions())
print(action_sorted)

[Card(♣6), Card(♣8), Card(♡8)]
[Card(♣6), Card(♣8), Card(♡8)]


## 評価値の検討

In [ ]:
# ランダムAIと対戦
state = State()
# 20ターン進める
for ppp in range(20):
    
    # ゲーム終了時
    if state.is_done():
        print("勝者 プレイヤー"+str(state.turn_player)+"番")
        break;
        
    pass_flag = 0
    # 行動の取得
    if state.turn_player == MY_PLAYER_NUM:
        action,pass_flag = my_AI(state)
        print(termcolor.colored(state, 'red'))
    else:
        action = random_action(state)
        print(state)

    # 次の状態の取得
    if pass_flag == 1:
        state = state.next(action,pass_flag)
    else:
        state = state.next(action)

場のカード

 --  --  --  --  --  --  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー0番　　パス回数0

あなたの手札
♢8 ♡A ♠6 ♢A ♠A ♡J ♡K ♣5 ♢Q ♡8 ♢K ♣A ♣Q ♠8 ♣2 ♠5 

出せるカード
♠6 ♠8 ♡8 ♢8 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー1番　　パス回数0

あなたの手札
♠10 ♡3 ♢5 ♡2 ♢J ♠4 ♢4 ♣6 ♡Q ♠3 ♣3 ♠2 ♣K ♠J ♣4 

出せるカード
♣6 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --  --  --  --  --  ♣6  ♣7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♡7  --  --  --  --  --  -- 
 --  --  --  --  --  --  ♢7  --  --  --  --  --  -- 

プレイヤー2番　　パス回数0

あなたの手札
♠K ♣8 ♣J ♡10 ♢3 ♡6 ♠Q ♢9 ♢10 ♢6 ♠9 ♡5 ♡9 ♢2 ♣9 ♡4 ♣10 

出せるカード
♣8 ♡6 ♢6 

場のカード

 --  --  --  --  --  ♠6  ♠7  --  --  --  --  --  -- 
 --  -

In [ ]:
print(state)

場のカード

 --  --  ♠3  ♠4  ♠5  ♠6  ♠7  ♠8  ♠9  ♠10  ♠J  --  -- 
 --  --  --  --  ♣5  ♣6  ♣7  --  --  --  --  --  -- 
 --  --  --  --  ♡5  ♡6  ♡7  ♡8  ♡9  --  --  --  -- 
 --  --  ♢3  ♢4  ♢5  ♢6  ♢7  ♢8  --  --  --  --  -- 

プレイヤー2番　　パス回数0

あなたの手札
♠K ♣8 ♣J ♡10 ♠Q ♢9 ♢10 ♢2 ♣9 ♡4 ♣10 

出せるカード
♠Q ♣8 ♡4 ♡10 ♢2 ♢9 



In [ ]:
state.field_cards

array([[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])

In [ ]:
state.field_cards[0].sum()

9

In [ ]:
state.field_cards[1].sum()

3

In [ ]:
temp = np.array([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])

In [ ]:
up = state.field_cards*temp
state.field_cards*temp

array([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [ ]:
up[0].sum()

4

### 1.手札に関する関数

In [ ]:
 #リストで手札を表示する　
state.my_hands()

[Card(♡3),
 Card(♣5),
 Card(♠8),
 Card(♣4),
 Card(♣8),
 Card(♡2),
 Card(♣10),
 Card(♡J),
 Card(♠J),
 Card(♠4),
 Card(♠A),
 Card(♡10)]

In [ ]:
#リストで手札の数字を表示する
state.my_hands().check_number()

[3, 5, 8, 4, 8, 2, 10, 11, 11, 4, 1, 10]

In [ ]:
#リストで手札のマークを表示する
state.my_hands().check_suit()

['♡', '♣', '♠', '♣', '♣', '♡', '♣', '♡', '♠', '♠', '♠', '♡']

In [ ]:
#リストで自分が出せるカードを表示する
state.my_actions()

[Card(♠8), Card(♣8)]

In [ ]:
#リストで自分が出せるカードの数字を表示する
Hand(state.my_actions()).check_number()

[8, 8]

In [ ]:
#リストで自分が出せるカードの記号を表示する
Hand(state.my_actions()).check_suit()

['♠', '♣']

### 2.場の札に関する関数

In [ ]:
#場のカードを表示する
state.field_cards

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])

In [ ]:
state.field_cards[0,6]

1

In [ ]:
#場で出せるカードをリストで取得する
state.legal_actions()

[Card(♠6),
 Card(♠8),
 Card(♣6),
 Card(♣8),
 Card(♡6),
 Card(♡8),
 Card(♢6),
 Card(♢8)]

In [ ]:
#場で出せるカードの数字をリストで取得する
Hand(state.legal_actions()).check_number()

[6, 8, 6, 8, 6, 8, 6, 8]

In [ ]:
#場で出せるカードの記号をリストで取得する
Hand(state.legal_actions()).check_suit()

['♠', '♠', '♣', '♣', '♡', '♡', '♢', '♢']

 ### 3.状態に関する関数

In [ ]:
#今のプレイヤーの番号を表示する
state.turn_player

0

In [ ]:
#3回パスをしてしまったプレイヤーを表示する
state.out_player

[]